<a href="https://colab.research.google.com/github/Shuichiro-T/gijutsushoten7_raspi_with_edgetpu_programs/blob/master/chap3/EdgeWithRaspi3v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tensorflowを1.14にダウングレードする。インストール後ランタイムの再起動を求められるので再起動する。

In [1]:
!pip install tensorflow==1.14

ライブラリのインポートを行う。

In [2]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

MNISTデータのダウンロードを行い、画像データの正規化する（０～１の範囲にする）ために255で割る。

In [3]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

学習モデルの生成を行う。

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


学習と評価を行う。

In [5]:
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5
60000/60000 [==============================] - 10s 164us/sample - loss: 0.2228 - acc: 0.9346
Epoch 2/5
60000/60000 [==============================] - 9s 157us/sample - loss: 0.0954 - acc: 0.9705
Epoch 3/5
60000/60000 [==============================] - 9s 155us/sample - loss: 0.0691 - acc: 0.9779
Epoch 4/5
60000/60000 [==============================] - 9s 152us/sample - loss: 0.0538 - acc: 0.9830
Epoch 5/5
10000/10000 [==============================] - 1s 59us/sample - loss: 0.0689 - acc: 0.9795


[0.06890781578962923, 0.9795]

モデルを保存するためにGoogle Driveのアクセス権を確認する。出力されたURLへアクセスし、認証を行った後に出力されるコードをコピーしダイアログへ入力する。

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


学習したモデルをGoogle Driveへ出力する。

In [8]:
model.save(
    '/content/drive/My Drive/Colab Notebooks/mymnist.h5')

出力したモデルを量子化しTensorFlow Liteのモデルへ変換する。

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(
    '/content/drive/My Drive/Colab Notebooks/mymnist.h5')
converter.inference_type = tf.lite.constants.QUANTIZED_UINT8
converter.default_ranges_stats = (0, 255)
input_arrays = converter.get_input_arrays()
converter.quantized_input_stats = {input_arrays[0] : (0., 1.)} 
tflite_model = converter.convert()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


TensorFlow LiteのモデルをGoogle Driveへ保存する。

In [10]:
open('/content/drive/My Drive/Colab Notebooks/mymnist.tflite', 
     "wb").write(tflite_model)


410072

Edge TPUコンパイラをインストールする。

In [17]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg \
 | sudo apt-key add -
!echo \
 "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" \
 | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get
!sudo apt-get install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1974  100  1974    0     0  89727      0 --:--:-- --:--:-- --:--:-- 89727
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
apt 1.6.12ubuntu0.2 (amd64)
Usage: apt-get [options] command
       apt-get [options] install|remove pkg1 [pkg2 ...]
       apt-get [options] source pkg1 [pkg2 ...]

apt-get is a command line interface for retrieval of packages
and information about them from authenticated sources and
for installation, upgrade and removal of packages together
with their dependencies.

Most used commands:
  update - Retrieve new lists of packages
  upgrade - Perform an upgrade
  install - Install new packages (pkg is libc6 not libc6.deb)
  remove - Remove packages
  purge - Remove packages and config files
  autoremove - Remove automatically all unused packages
  dist-upgrade - Distribution upgrade, s

Edge TPU用に変換をする。

In [18]:
!edgetpu_compiler \
 "/content/drive/My Drive/Colab Notebooks/mymnist.tflite" \
 -o "/content/drive/My Drive/Colab Notebooks/" \
 -m 10

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 87 ms.

Input model: /content/drive/My Drive/Colab Notebooks/mymnist.tflite
Input size: 400.46KiB
Output model: /content/drive/My Drive/Colab Notebooks/mymnist_edgetpu.tflite
Output size: 452.53KiB
On-chip memory used for caching model parameters: 427.75KiB
On-chip memory remaining for caching model parameters: 7.44MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 3
Operation log: /content/drive/My Drive/Colab Notebooks/mymnist_edgetpu.log
See the operation log file for individual operation details.
